In [1]:
library(ggplot2)
library(mgcv)
library(visreg)
library(reshape2)
library(ggpubr)
library(ggpattern)
library(plyr)
library(ggsignif)
library(lmerTest)

datax=read.csv('gmdata2.csv')
datax = na.omit(datax)
datax = datax[datax$CBF.GM < 120 ,]
datax = datax[datax$CBF.GM > 5 ,]

Loading required package: nlme

This is mgcv 1.8-36. For overview type 'help("mgcv-package")'.

Loading required namespace: memoise


Attaching package: ‘plyr’


The following object is masked from ‘package:ggpubr’:

    mutate


Loading required package: lme4

Loading required package: Matrix


Attaching package: ‘lme4’


The following object is masked from ‘package:nlme’:

    lmList



Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step




In [3]:
datagm = datax 
datagm$CBF.WM =NULL
datagm$TP ='GM'
datagm$CBF=datagm$CBF.GM 
datagm$CBF.GM = NULL 

datawm = datax 
datawm$CBF.GM =NULL
datawm$TP ='WM'
datawm$CBF=datawm$CBF.WM  
datawm$CBF.WM = NULL 

datasets = rbind(datagm,datawm)

datasetsy = datasets[datasets$CBFTYPE =='SCBF',]
datasetsy = na.omit(datasetsy)
dodge <- position_dodge(width = 0.5)
dp <- ggplot(datasetsy, aes(x=Datasets,y=CBF,fill=TP),plot = FALSE) + 
  geom_violin(position = dodge,width = 1.4)+
  geom_boxplot(width=.1, outlier.colour=NA, position = dodge) 
dp=dp + theme_classic() + scale_x_discrete(limits = rev) + 
  labs(x = "Datasets", y = "CBF(mL/100 g/min)") +
  theme(axis.title.x = element_text(size = rel(1.2))) +
  theme(axis.title.y = element_text(size = rel(1.2) ,vjust=-0.6)) + ylim(0,140) +
  theme(axis.text = element_text(size = rel(1.4))) + theme(axis.line = element_line(colour = 'black', size = 1.5)) +
  scale_fill_manual(values=c("#d95f0e","#756bb1")) + theme(legend.title = element_blank())

#png(filename="CBF_GM_WM.png",res = 600, width = 6, height = 4, units = 'in')
#print(dp)







datay = datax[datax$CBFTYPE =='SCBF',]
pnc = datay[datay$Datasets=='PNC',]
nki = datay[datay$Datasets=='NKI',]
age = datay[datay$Datasets=='AGE',]
irr = datay[datay$Datasets=='IRR',]
ftd = datay[datay$Datasets=='FTD',]

#############################################################
cbf_Age_gam <- gam(CBF.GM ~ s(AGE, k=4), method="REML", data = datay)

#####################
## Look at results ##
#####################
summary(cbf_Age_gam)

## Nonlinear age effect
Age_pval <- summary(cbf_Age_gam)$s.table[1,4]
Age_pval

####################################
## Visualize Nonlinear Age Effect ##
####################################
plotdata <- visreg(cbf_Age_gam,'AGE',type = "conditional",scale = "linear", plot = FALSE)
smooths <- data.frame(Variable = plotdata$meta$x, 
                      x=plotdata$fit[[plotdata$meta$x]], 
                      smooth=plotdata$fit$visregFit, 
                      lower=plotdata$fit$visregLwr, 
                      upper=plotdata$fit$visregUpr)
predicts <- data.frame(Variable = "dim1", 
                       x=plotdata$res$AGE,
                       y=plotdata$res$visregRes)

CBF_Age_plot <- ggplot() + xlim(8, 85)+ ylim(0,140) +
  #theme(legend.position = "none")  +
  labs(x = "Age (years)", y = "CBF(mL/100 g/min)",) +
  theme(axis.title.x = element_text(size = rel(1.2))) + 
  theme(axis.title.y = element_blank(),axis.ticks.y = element_blank(),axis.text.y = element_blank()) +
  theme(axis.text = element_text(size = rel(1.2))) + theme(axis.line = element_line(colour = 'black', size = 1.5), axis.ticks.length = unit(.25, "cm")) +
  theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(), panel.background = element_blank()) +
  geom_point(data = predicts, aes(x, y), colour = "darksalmon", alpha=0.7 ) +  
  #geom_point(data=datay,aes(x=AGE, y=CBF.GM),size=1)+ 
  geom_point(data=pnc,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill='#e34a33', color="white")+ 
  geom_point(data=nki,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill="#3182bd", color="white")+ 
  geom_point(data=irr,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill="#31a354", color="white")+ 
  geom_point(data=ftd,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill="#d95f02", color="white")+ 
  geom_point(data=age,aes(x=AGE, y=CBF.GM),shape=21,size=2,fill="#c51b8a", color="white")+ 
  geom_line(data = smooths, aes(x = x, y = smooth), colour = "midnightblue",size=1) +
  geom_line(data = smooths, aes(x = x, y=lower), linetype="dashed", colour = "midnightblue", alpha = 0.9, size = 0.9) + 
  geom_line(data = smooths, aes(x = x, y=upper), linetype="dashed",colour = "midnightblue", alpha = 0.9, size = 0.9) +
  coord_cartesian(xlim = c(10.5,85), ylim = c(0,140))
  

figure <- ggarrange(dp,CBF_Age_plot,
                    ncol = 2, nrow = 1,widths=c(1.5,1) )
ggsave("fig2.png", width=7, units="in", height=3,dpi=800)


Family: gaussian 
Link function: identity 

Formula:
CBF.GM ~ s(AGE, k = 4)

Parametric coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   61.937      0.227   272.9   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Approximate significance of smooth terms:
         edf Ref.df    F p-value    
s(AGE) 2.993      3 1189  <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

R-sq.(adj) =   0.51   Deviance explained = 51.1%
-REML =  13711  Scale est. = 176.31    n = 3422

[1] 0

Warning message:
“Removed 1 rows containing non-finite values (stat_ydensity).”
Warning message:
“Removed 1 rows containing non-finite values (stat_boxplot).”
Warning message:
“position_dodge requires non-overlapping x intervals”
Warning message:
“Removed 8 rows containing missing values (geom_point).”
Warning message:
“Removed 8 rows containing missing values (geom_point).”
Warning message:
“Removed 3 row(s) containing missing values (geom_path).”
Warning message:
“Removed 3 row(s) containing missing values (geom_path).”
Warning message:
“Removed 3 row(s) containing missing values (geom_path).”
